In [1]:
!pip install natsort

In [2]:
#### import os
import torch
import cv2
import random
from PIL import Image
# import numpy as np
from torchvision import transforms
import glob as gl
from torch.utils import data
import natsort
#读取完成的图像
#training.....
class ImagePools(object):
    def __init__(self, root='',trans=None,mode=False,high=False):
        super().__init__()
        self.mode=mode
        self.high=high
        self.transform = transforms.Compose(trans)
        if not self.mode:
            
            self.A_path = os.path.join(root, "train_photo/*")
            self.B_path = os.path.join(root, "hayao/*")
        elif self.high:
            self.A_path = os.path.join(root, "testA/*")
            self.B_path = os.path.join(root, "testB/*")
        else:    
            self.A_path = os.path.join(root, "train_photo/*")
            self.B_path = os.path.join(root, "hayao/*")
        # 读取图像文件
#         self.list_A = natsort.natsorted(sorted(make_dataset(self.A_path,1000)))
#         self.list_B = natsort.natsorted(sorted(make_dataset(self.B_path,1000)))
        self.list_A = gl.glob(self.A_path)
        self.list_B = gl.glob(self.B_path)

    def __getitem__(self, index):
        data={}
        A_path = self.list_A[index]
        B_path = random.choice(self.list_B)
        A = Image.open(A_path).convert('RGB')
        B = Image.open(B_path).convert('RGB')
        A = self.transform(A)
        B = self.transform(B)
        data.update({"A":A,"B":B})
        return A,B

    def __len__(self):
        return max(len(self.list_A),len(self.list_B))

#testing.....
# class ImagePools(object):
#     def __init__(self, root='',trans=None,mode=False,high=False):
#         super().__init__()
#         self.mode=mode
#         self.high=high
#         self.transform = transforms.Compose(trans)
#         if not self.mode:
#             self.A_path = os.path.join(root, "testHR")
#             self.B_path = os.path.join(root, "hayao")
            
# #             self.A_path = os.path.join(root, "train_photo")
# #             self.B_path = os.path.join(root, "hayao")
# #         elif self.high:
# #             self.A_path = os.path.join(root, "testA/*")
# #             self.B_path = os.path.join(root, "testB/*")
        
#         # 读取图像文件
#         self.list_A = natsort.natsorted(sorted(make_dataset(self.A_path,1000)))
#         self.list_B = natsort.natsorted(sorted(make_dataset(self.B_path,1000)))
# #         self.list_A = gl.glob(self.A_path)
# #         self.list_B = gl.glob(self.B_path)

#     def __getitem__(self, index):
#         data={}
#         A_path = self.list_A[index]
#         B_path = random.choice(self.list_B)
#         A = Image.open(A_path).convert('RGB')
#         B = Image.open(B_path).convert('RGB')
#         A = self.transform(A)
#         B = self.transform(B)
#         data.update({"A":A,"B":B})
#         return A,B

#     def __len__(self):
# #         return max(len(self.list_A),len(self.list_B))
#         return min(len(self.list_A),len(self.list_B))

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
    '.tif', '.TIF', '.tiff', '.TIFF',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset(dir, max_dataset_size=float("inf")):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir
    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)
    return images[:min(max_dataset_size, len(images))]

In [3]:
import torch
from torch import nn
import numpy as np
import cv2
import os
from torch.autograd import Variable

   
#参数权重初始化
def init_weights(m, init_type='normal', gain=0.02):
    classname = m.__class__.__name__
    if classname.find('BatchNorm2d') != -1:
        if hasattr(m, 'weight') and m.weight is not None:
            torch.nn.init.normal_(m.weight.data, 1.0, gain)
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
        if init_type == 'normal':
            torch.nn.init.normal_(m.weight.data, 0.0, gain)
        elif init_type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight.data, gain=gain)
        elif init_type == 'xavier_uniform':
            torch.nn.init.xavier_uniform_(m.weight.data, gain=1.0)
        elif init_type == 'kaiming':
            torch.nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
        elif init_type == 'orthogonal':
            torch.nn.init.orthogonal_(m.weight.data, gain=gain)
        elif init_type == 'none':  # uses pytorch's default init method
            m.reset_parameters()
        else:
            raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
#梯度
def requires_grad(model, flag=True):
    if model is None:
        return
    for p in model.parameters():
        p.requires_grad = flag

def rgb_to_yuv(image,x):
    image = (image + 1.0) / 2.0
    yuv_img = torch.tensordot(
        image,
        x,
        dims=([image.ndim - 3], [0]))
    return yuv_img
#时间转化
def time_change(time):
    new_time = t.localtime(time)
    new_time = t.strftime("%Hh%Mm%Ss", new_time)
    return new_time
#归一化
def  denorm(x):
    x=(x* 0.5+ 0.5)*255.0
    return x.cpu().detach().numpy().transpose(1,2,0)
def  process(x):
    x=(x* 0.5+ 0.5)*255.0
    return x.cpu().detach().numpy()[0][0]
def RGB2BGR(x):
    return cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
#创建文件目录
def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir
#颜色RGB->Gray(DCT-NET)
def color_shift(image, mode='uniform'):
    device = image.device
    b1, g1, r1 = torch.split(image, 1, dim=1)
    if mode == 'normal':
        b_weight = torch.normal(mean=0.114, std=0.1, size=[1]).to(device)
        g_weight = torch.normal(mean=0.587, std=0.1, size=[1]).to(device)
        r_weight = torch.normal(mean=0.299, std=0.1, size=[1]).to(device)
    elif mode == 'uniform':
        b_weight = torch.FloatTensor(1).uniform_(0.014, 0.214).to(device)
        g_weight = torch.FloatTensor(1).uniform_(0.487, 0.687).to(device)
        r_weight = torch.FloatTensor(1).uniform_(0.199, 0.399).to(device)
    output1 = (b_weight * b1 + g_weight * g1 + r_weight * r1) / (b_weight + g_weight + r_weight)
    return output1
#图像pool
class Queue():
    def __init__(self, pool_size=50):
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        if self.pool_size == 0:
            return images
        return_images = []
        for image in images.data:
            image = torch.unsqueeze(image, 0)
            if self.num_imgs < self.pool_size:
                self.num_imgs = self.num_imgs + 1
                self.images.append(image)
                return_images.append(image)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.pool_size-1)
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:
                    return_images.append(image)
        return_images = Variable(torch.cat(return_images, 0))
        return return_images
#颜色white-box
class ColorShift():
    def __init__(self, device: torch.device = 'cuda', mode='uniform', image_format='rgb'):
        self.dist: torch.distributions = None
        self.dist_param1: torch.Tensor = None
        self.dist_param2: torch.Tensor = None

        if (mode == 'uniform'):
            self.dist_param1 = torch.tensor((0.199, 0.487, 0.014), device=device)
            self.dist_param2 = torch.tensor((0.399, 0.687, 0.214), device=device)
            if (image_format == 'bgr'):
                self.dist_param1 = torch.permute(self.dist_param1, (2, 1, 0))
                self.dist_param2 = torch.permute(self.dist_param2, (2, 1, 0))

            self.dist = torch.distributions.Uniform(low=self.dist_param1, high=self.dist_param2)


        elif (mode == 'normal'):
            self.dist_param1 = torch.tensor((0.299, 0.587, 0.114), device=device)
            self.dist_param2 = torch.tensor((0.1, 0.1, 0.1), device=device)
            if (image_format == 'bgr'):
                self.dist_param1 = torch.permute(self.dist_param1, (2, 1, 0))
                self.dist_param2 = torch.permute(self.dist_param2, (2, 1, 0))

            self.dist = torch.distributions.Normal(loc=self.dist_param1, scale=self.dist_param2)

    # Allow taking mutiple images batches as input
    # So we can do: gray_fake, gray_cartoon = ColorShift(output, input_cartoon)
    def process(self, *image_batches: torch.Tensor):
        # Sample the random color shift coefficients
        weights = self.dist.sample()

        # images * weights[None, :, None, None] => Apply weights to r,g,b channels of each images
        # torch.sum(, dim=1) => Sum along the channels so (B, 3, H, W) become (B, H, W)
        # .unsqueeze(1) => add back the channel so (B, H, W) become (B, 1, H, W)
        # .repeat(1, 3, 1, 1) => (B, 1, H, W) become (B, 3, H, W) again
        return (
        (((torch.sum(images * weights[None, :, None, None], dim=1)) / weights.sum()).unsqueeze(1)).repeat(1, 3, 1, 1)
        for images in image_batches)

#patch 抽取
def extract_image_patches(x, kernel, stride):

        if kernel != 1:
            x = nn.ZeroPad2d(1)(x)
        x = x.permute(0, 2, 3, 1)
        all_patches = x.unfold(1, kernel, stride).unfold(2, kernel, stride)
        all_patches = all_patches.permute(0, 3, 1, 2, 4, 5)
        all_patches = all_patches.reshape([all_patches.shape[0] * all_patches.shape[2] ** 2, all_patches.shape[1],
                                           all_patches.shape[4], all_patches.shape[5]])
        return all_patches
def gram(input):
    b, c, w, h = input.size()
    x = input.view(b * c, w * h)
    G = torch.mm(x, x.T)
    return G.div(b * c * w * h)

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    return num_params
def high_pass_filter(img, d, n):
    return (1 - 1 / (1 + (img / d) ** n))

#引导滤波
class GuidedFilter():
    def box_filter(self, x, r):
        channel =  x.shape[1] # Batch, Channel, H, W
        kernel_size = (2*r+1)
        weight = 1.0/(kernel_size**2)
        box_kernel = weight*torch.ones((channel, 1, kernel_size, kernel_size), dtype=torch.float32, device=x.device)
        output = F.conv2d(x, weight=box_kernel, stride=1, padding=r, groups=channel) #tf.nn.depthwise_conv2d(x, box_kernel, [1, 1, 1, 1], 'SAME')
        return output
    def guided_filter(self, x, y, r, eps=1e-2):
        # Batch, Channel, H, W
        _, _, H, W = x.shape

        N = self.box_filter(torch.ones((1, 1, H, W), dtype=x.dtype, device=x.device), r)

        mean_x = self.box_filter(x, r) / N
        mean_y = self.box_filter(y, r) / N
        cov_xy = self.box_filter(x * y, r) / N - mean_x * mean_y
        var_x  = self.box_filter(x * x, r) / N - mean_x * mean_x

        A = cov_xy / (var_x + eps)
        b = mean_y - A * mean_x

        mean_A = self.box_filter(A, r) / N
        mean_b = self.box_filter(b, r) / N

        output = mean_A * x + mean_b
        return output

#tv_loss
class VariationLoss(nn.Module):
    def __init__(self, k_size: int) -> None:
        super().__init__()
        self.k_size = k_size

    def forward(self, image: torch.Tensor):
        b, c, h, w = image.shape
        tv_h = torch.mean((image[:, :, self.k_size:, :] - image[:, :, : -self.k_size, :])**2)
        tv_w = torch.mean((image[:, :, :, self.k_size:] - image[:, :, :, : -self.k_size])**2)
        tv_loss = (tv_h + tv_w) / (3 * h * w)
        return tv_loss

def get_gaussian_kernel(k=3, mu=0, sigma=1, normalize=True):
   # compute 1 dimension gaussian
   gaussian_1D = np.linspace(-1, 1, k)
   # compute a grid distance from center
   x, y = np.meshgrid(gaussian_1D, gaussian_1D)
   distance = (x ** 2 + y ** 2) ** 0.5

   # compute the 2 dimension gaussian
   gaussian_2D = np.exp(-(distance - mu) ** 2 / (2 * sigma ** 2))
   gaussian_2D = gaussian_2D / (2 * np.pi * sigma ** 2)

   # normalize part (mathematically)
   if normalize:
       gaussian_2D = gaussian_2D / np.sum(gaussian_2D)
   return gaussian_2D


def get_sobel_kernel(k=3):
   # get range
   range = np.linspace(-(k // 2), k // 2, k)
   # compute a grid the numerator and the axis-distances
   x, y = np.meshgrid(range, range)
   sobel_2D_numerator = x
   sobel_2D_denominator = (x ** 2 + y ** 2)
   sobel_2D_denominator[:, k // 2] = 1  # avoid division by zero
   sobel_2D = sobel_2D_numerator / sobel_2D_denominator
   return sobel_2D


def get_thin_kernels(start=0, end=360, step=45):
   k_thin = 3  # actual size of the directional kernel
   # increase for a while to avoid interpolation when rotating
   k_increased = k_thin + 2

   # get 0° angle directional kernel
   thin_kernel_0 = np.zeros((k_increased, k_increased))
   thin_kernel_0[k_increased // 2, k_increased // 2] = 1
   thin_kernel_0[k_increased // 2, k_increased // 2 + 1:] = -1

   # rotate the 0° angle directional kernel to get the other ones
   thin_kernels = []
   for angle in range(start, end, step):
       (h, w) = thin_kernel_0.shape
       # get the center to not rotate around the (0, 0) coord point
       center = (w // 2, h // 2)
       # apply rotation
       rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
       kernel_angle_increased = cv2.warpAffine(thin_kernel_0, rotation_matrix, (w, h), cv2.INTER_NEAREST)

       # get the k=3 kerne
       kernel_angle = kernel_angle_increased[1:-1, 1:-1]
       is_diag = (abs(kernel_angle) == 1)  # because of the interpolation
       kernel_angle = kernel_angle * is_diag  # because of the interpolation
       thin_kernels.append(kernel_angle)
   return thin_kernels

#Canny 边缘检测
class CannyFilter(nn.Module):
   def __init__(self,
                k_gaussian=3,
                mu=0,
                sigma=1,
                k_sobel=3,
                device = 'cuda:0'):
       super(CannyFilter, self).__init__()
       # device
       self.device = device
       # gaussian
       gaussian_2D = get_gaussian_kernel(k_gaussian, mu, sigma)
       self.gaussian_filter = nn.Conv2d(in_channels=1,
                                        out_channels=1,
                                        kernel_size=k_gaussian,
                                        padding=k_gaussian // 2,
                                        bias=False)
       self.gaussian_filter.weight.data[:,:] = nn.Parameter(torch.from_numpy(gaussian_2D), requires_grad=False)

       # sobel

       sobel_2D = get_sobel_kernel(k_sobel)
       self.sobel_filter_x = nn.Conv2d(in_channels=1,
                                       out_channels=1,
                                       kernel_size=k_sobel,
                                       padding=k_sobel // 2,
                                       bias=False)
       self.sobel_filter_x.weight.data[:,:] = nn.Parameter(torch.from_numpy(sobel_2D), requires_grad=False)

       self.sobel_filter_y = nn.Conv2d(in_channels=1,
                                       out_channels=1,
                                       kernel_size=k_sobel,
                                       padding=k_sobel // 2,
                                       bias=False)
       self.sobel_filter_y.weight.data[:,:] = nn.Parameter(torch.from_numpy(sobel_2D.T), requires_grad=False)

       # thin

       thin_kernels = get_thin_kernels()
       directional_kernels = np.stack(thin_kernels)

       self.directional_filter = nn.Conv2d(in_channels=1,
                                           out_channels=8,
                                           kernel_size=thin_kernels[0].shape,
                                           padding=thin_kernels[0].shape[-1] // 2,
                                           bias=False)
       self.directional_filter.weight.data[:, 0] = nn.Parameter(torch.from_numpy(directional_kernels), requires_grad=False)

       # hysteresis

       hysteresis = np.ones((3, 3)) + 0.25
       self.hysteresis = nn.Conv2d(in_channels=1,
                                   out_channels=1,
                                   kernel_size=3,
                                   padding=1,
                                   bias=False)
       self.hysteresis.weight.data[:,:] = nn.Parameter(torch.from_numpy(hysteresis), requires_grad=False)

   def forward(self, img, low_threshold=None, high_threshold=None, hysteresis=True):
       # set the setps tensors
       B, C, H, W = img.shape
       blurred = torch.zeros((B, C, H, W)).to(self.device)
       grad_x = torch.zeros((B, 1, H, W)).to(self.device)
       grad_y = torch.zeros((B, 1, H, W)).to(self.device)
       grad_magnitude = torch.zeros((B, 1, H, W)).to(self.device)
       grad_orientation = torch.zeros((B, 1, H, W)).to(self.device)

       # gaussian

       for c in range(C):
           blurred[:, c:c + 1] = self.gaussian_filter(img[:, c:c + 1])
           grad_x = grad_x + self.sobel_filter_x(blurred[:, c:c + 1])
           grad_y = grad_y + self.sobel_filter_y(blurred[:, c:c + 1])

       # thick edges

       grad_x, grad_y = grad_x / C, grad_y / C
       grad_magnitude = (grad_x ** 2 + grad_y ** 2) ** 0.5
       grad_orientation = torch.atan2(grad_y, grad_x)
       grad_orientation = grad_orientation * (180 / np.pi) + 180  # convert to degree
       grad_orientation = torch.round(grad_orientation / 45) * 45  # keep a split by 45

       # thin edges

       directional = self.directional_filter(grad_magnitude)
       # get indices of positive and negative directions
       positive_idx = (grad_orientation / 45) % 8
       negative_idx = ((grad_orientation / 45) + 4) % 8
       thin_edges = grad_magnitude.clone()
       # non maximum suppression direction by direction
       for pos_i in range(4):
           neg_i = pos_i + 4
           # get the oriented grad for the angle
           is_oriented_i = (positive_idx == pos_i) * 1
           is_oriented_i = is_oriented_i + (positive_idx == neg_i) * 1
           pos_directional = directional[:, pos_i]
           neg_directional = directional[:, neg_i]
           selected_direction = torch.stack([pos_directional, neg_directional])

           # get the local maximum pixels for the angle
           # selected_direction.min(dim=0)返回一个列表[0]中包含两者中的小的，[1]包含了小值的索引
           is_max = selected_direction.min(dim=0)[0] > 0.0
           is_max = torch.unsqueeze(is_max, dim=1)

           # apply non maximum suppression
           to_remove = (is_max == 0) * 1 * (is_oriented_i) > 0
           thin_edges[to_remove] = 0.0

       # thresholds

       if low_threshold is not None:
           low = thin_edges > low_threshold

           if high_threshold is not None:
               high = thin_edges > high_threshold
               # get black/gray/white only
               thin_edges = low * 0.5 + high * 0.5

               if hysteresis:
                   # get weaks and check if they are high or not
                   weak = (thin_edges == 0.5) * 1
                   weak_is_high = (self.hysteresis(thin_edges) > 1) * weak
                   thin_edges = high * 1 + weak_is_high * 1
           else:
               thin_edges = low * 1

       return thin_edges * 255
#掩码操作mask
def mask(img,mask,y1,y2,Canny):
    edge_fake_img = Canny(img, y1, y2)
    edge_real_img = Canny(mask, y1, y2)
    edge_fake_img=high_pass_filter(edge_fake_img,d=0.2, n=2)
    edge_real_img=high_pass_filter(edge_real_img,d=0.2, n=2)
    #转换
    edge_real_img=process(edge_real_img)
    edge_fake_img = process(edge_fake_img)
    result=cv2.bitwise_and( edge_fake_img,  edge_real_img)
    result=torch.from_numpy(result)
    edge_real_img=torch.from_numpy(edge_real_img)
    return result,edge_real_img
#计算均值
def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std

def calc_mean(feat):
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean

def nor_mean_std(feat):
    size = feat.size()
    mean, std = calc_mean_std(feat)
    nor_feat = (feat - mean.expand(size)) / std.expand(size)
    return nor_feat

def nor_mean(feat):
    size = feat.size()
    mean = calc_mean(feat)
    nor_feat = feat - mean.expand(size)
    return nor_feat, mean    

def calc_cov(feat):
    feat = feat.flatten(2, 3)
    f_cov = torch.bmm(feat, feat.permute(0,2,1)).div(feat.size(2))
    return f_cov 

In [4]:
import cv2
import torch
from torch import nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.nn.utils import spectral_norm
from torchvision import transforms
# from torchvision.transforms import InterpolationMode
from torch.utils import data
#channle_wise
class layer_norm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return F.layer_norm(x, x.size()[1:])
class Conv2D(nn.Module):
    def __init__(self,in_channels=256,out_channels=256, kernel_size=3, strides=1,padding=1,sn=False):
        super().__init__()
        self.sn=sn
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=strides, padding=padding,
                     padding_mode="reflect")

    def forward(self,x):
        return self.conv(x)
#定义卷积
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, strides=1, padding=0,sn=False, bias=False,padding_mode='reflect'):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,
            kernel_size=kernel_size, stride=strides, padding=padding, bias=bias,padding_mode=padding_mode)
        self.ins_norm= layer_norm()
        self.activation = nn.LeakyReLU(0.2, True)

        init_weights(self)

    def forward(self, x):
        out = self.conv(x)
        out = self.ins_norm(out)
        out = self.activation(out)

        return out
    
class Discriminator_T(nn.Module):
    def __init__(self):
        super( Discriminator_T,self).__init__()
        self.channels=[1,32,64,128,256]
        self.model=[]
        self.model+=[ConvBlock(in_channels=self.channels[0],out_channels=self.channels[1],padding=1,sn=True),
                     ConvBlock(in_channels=self.channels[1],out_channels=self.channels[2],strides=2,padding=1,sn=True),
                     ConvBlock(in_channels=self.channels[2],out_channels=self.channels[3],strides=2,padding=1,sn=True),
                     ConvBlock(in_channels=self.channels[3],out_channels=self.channels[4],strides=2,padding=1,sn=True),
                     Conv2D(in_channels=self.channels[4],out_channels=1,padding=1,sn=True)]
        self.model=nn.Sequential(*self.model)
    def forward(self,x):
        #3*1
        out=self.model(x)
        return out
#定义生成器
class ILN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(ILN, self).__init__()
        self.eps = eps
        self.rho = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.gamma = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.beta = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.rho.data.fill_(0.0)
        self.gamma.data.fill_(1.0)
        self.beta.data.fill_(0.0)

    def forward(self, input):
        in_mean, in_var = torch.mean(input, dim=[2, 3], keepdim=True), torch.var(input, dim=[2, 3], keepdim=True)
        out_in = (input - in_mean) / torch.sqrt(in_var + self.eps)
        ln_mean, ln_var = torch.mean(input, dim=[1, 2, 3], keepdim=True), torch.var(input, dim=[1, 2, 3], keepdim=True)
        out_ln = (input - ln_mean) / torch.sqrt(ln_var + self.eps)
        out = self.rho.expand(input.shape[0], -1, -1, -1) * out_in + (1-self.rho.expand(input.shape[0], -1, -1, -1)) * out_ln
        out = out * self.gamma.expand(input.shape[0], -1, -1, -1) + self.beta.expand(input.shape[0], -1, -1, -1)
        return out
#残差的SCT
class utm(nn.Module):
    def __init__(self):
        super(utm, self).__init__()

        self.net = nn.Sequential(nn.Conv2d(256,128,1,1,0),
                nn.ReLU(inplace=True),
                nn.Conv2d(128,32,1,1,0))
        self.uncompress = nn.Conv2d(32,256,1,1,0)  
        self.sm = nn.Softmax(dim=-1)
    def forward(self, content, style=None,noise=None,init=False):
        if init:
            cF_nor = nor_mean_std(content)
            cF = self.net(cF_nor)
            cF = self.uncompress(cF)
            cF = cF +content
            return cF
        else:
            cF_nor = nor_mean_std(content)
            sF_nor, smean = nor_mean(style)
            cF = self.net(cF_nor)
            sF = self.net(sF_nor)
            b, c, w, h = cF.size()
            s_cov = calc_cov(sF)
            b1, c1, hw = s_cov.size()
            s_cov = self.sm(s_cov) * int(c1) ** (-0.5)#test
            gF = torch.bmm(s_cov, cF.flatten(2, 3)).view(b,c,w,h)
            gF = self.uncompress(gF)
            if noise==None:
                gF = gF + smean.expand(cF_nor.size())+content
            else:
                gF = gF + smean.expand(cF_nor.size())+content
            return gF
# 定义残差块
class ResBlock(nn.Module):
    def __init__(self, channels, use_bias=False):
        super().__init__()
        Res_block = []
        Res_block += [nn.ReflectionPad2d(1),
                      nn.Conv2d(channels, channels, 3, 1, 0, bias=use_bias),
                      ILN(channels), nn.PReLU(num_parameters=1)]

        Res_block += [nn.ReflectionPad2d(1),
                      nn.Conv2d(channels, channels, 3, 1, 0, bias=use_bias),
                      ILN(channels)]
        self.Res_block = nn.Sequential(*Res_block)

    def forward(self, x):
        return x + self.Res_block(x)

class StyleEncoder(nn.Module):
    def __init__(self, img_channels=3, num_features=64, padding_mode="reflect",):
        super().__init__()
        self.padding_mode = padding_mode

        self.initial_down = nn.Sequential(
            # k7n32s1
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3,
                      padding_mode=self.padding_mode),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
        )

        # Down-convolution
        self.down1 = nn.Sequential(
            # k3n32s2
            nn.Conv2d(num_features, num_features, kernel_size=3, stride=2, padding=1,
                      padding_mode=self.padding_mode),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            # k3n64s1
            nn.Conv2d(num_features, num_features * 2, kernel_size=3, stride=1, padding=1,
                      padding_mode=self.padding_mode),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
        )

        self.down2 = nn.Sequential(
            # k3n64s2
            nn.Conv2d(num_features * 2, num_features * 2, kernel_size=3, stride=2, padding=1,
                      padding_mode=self.padding_mode),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            # k3n128s1
            nn.Conv2d(num_features * 2, num_features * 4, kernel_size=3, stride=1, padding=1,
                      padding_mode=self.padding_mode),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
           )
    def forward(self, x):
        style_code= self.down2(self.down1(self.initial_down(x)))
        
        return  style_code
# 定义生成器-Encode
class GenEncoder(nn.Module):
    def __init__(self, hw=64, n_block=None,norm=nn.InstanceNorm2d, use_bias=False):
        super().__init__()
        # 平面卷积
        model = []
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(3, hw, 7, 1, 0, bias=use_bias)]
        # 下采样
        down = 2
        for i in range(down):
            mult = 2 ** i
            model += [nn.Conv2d(hw * mult, hw * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                     ILN(hw * mult * 2), nn.ReLU(True)]
            # 残差块
        res = hw * 4
        for j in range(n_block):
            model += [ResBlock(res)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)  # 1,256,64,64


# 定义解码器
class GenDecoder(nn.Module):
    def __init__(self, hw=64, out_channels=3, n_block=None, use_bias=False):
        super().__init__()
        # 残差块
        model = []
        res = hw * 4
        for i in range(n_block):
            model += [ResBlock(res)]
        # frist upsample
        mult = 2 ** (n_block // 2)
        model += [nn.Upsample(scale_factor=2, mode='bilinear',align_corners=True),
                  nn.Conv2d(int(hw * mult), int(hw * mult / 2), kernel_size=3, stride=1, padding=1),
                  nn.Conv2d(int(hw * mult / 2), int(hw * mult / 2), kernel_size=3, stride=1, padding=1),
                  ILN(int(hw * mult / 2)),
                  nn.ReLU(True)]
        # second upsampling
        model += [nn.Upsample(scale_factor=2, mode='bilinear',align_corners=True),
                  nn.Conv2d(int(hw * mult / 2), int(hw * mult / 4), kernel_size=3, stride=1, padding=1),
                  nn.Conv2d(int(hw * mult / 4), int(hw * mult / 4), kernel_size=3, stride=1, padding=1),
                  ILN(int(hw * mult / 4)),
                  nn.ReLU(True)]
        # addtional layer
        model += [nn.Conv2d(int(hw * mult / 4), int(hw * mult / 8), kernel_size=3, stride=1, padding=1),
                  nn.Conv2d(int(hw * mult / 8), int(hw * mult / 8), kernel_size=3, stride=1, padding=1),
                  ILN(int(hw * mult / 8)), nn.ReLU(True)]
        model += [nn.Conv2d(int(hw * mult / 8), int(hw * mult / 16), kernel_size=3, stride=1, padding=1),
                  nn.Conv2d(int(hw * mult / 16), int(hw * mult / 16), kernel_size=3, stride=1, padding=1),
                  ILN(int(hw * mult / 16)), nn.ReLU(True)]
        model += [nn.Conv2d(int(hw * mult / 16), out_channels, 7, 1, 3), nn.Tanh()]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)
    # 定义生成器


#定义生成器               
#定义生成器
class Generator(nn.Module):
        def __init__(self,n_domian=None,E_block=5,D_block=4):
                  super(Generator,self).__init__()
                  #编码器
                  self.Encoder=[GenEncoder(n_block=E_block)]
                  self.Encoder=nn.Sequential(*self.Encoder)
                  #解码器
                  self.Decoder=[GenDecoder(n_block=D_block)]
                  self.Decoder=nn.Sequential(*self.Decoder)
        def encoder(self,x):
                    return self.Encoder(x)  # type: ignore
        def decoders(self,x):
                    
                return self.Decoder(x)
                    
        def forward(self,x):
                encode=self.encoder(x)
                return self.decoders(encode)


#img _level D
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, act=True):
        super().__init__()
        self.act = act
        self.sn_conv = spectral_norm(nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride, 
                padding,
                padding_mode="zeros" # Author's code used slim.convolution2d, which is using SAME padding (zero padding in pytorch) 
            ))
        self.LReLU = nn.LeakyReLU(negative_slope=0.2, inplace=True)
    def forward(self, x):
        x = self.sn_conv(x)
        if self.act:
            x = self.LReLU(x)
        return x
class Discriminator_S(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[32, 64, 128,256]):
        super().__init__()
        
        self.model = nn.Sequential(
           
            #k3n32s2
            Block(in_channels, features[0], kernel_size=3, stride=2, padding=1),#b,32,128,128
            #k3n32s1
            Block(features[0], features[0], kernel_size=3, stride=1, padding=1),#b,32,128,128

            #k3n64s2
            Block(features[0], features[1], kernel_size=3, stride=2, padding=1),#b,64,64,64
            #k3n64s1
            Block(features[1], features[1], kernel_size=3, stride=1, padding=1),#b,64,64,64

            #k3n128s2
            Block(features[1], features[2], kernel_size=3, stride=2, padding=1),#b,128,32,32
            #k3n128s1
            Block(features[2], features[2], kernel_size=3, stride=1, padding=1),#b,128,32,32

            #k1n1s1
            Block(features[2], out_channels, kernel_size=1, stride=1, padding=0, act=False)#b,3,32,32
        )
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    
# #patchgan  
# class PatchGAN_D(nn.Module):
#           def __init__(self,hw=64,out_channels=1,norm=nn.InstanceNorm2d,use_bias=False,use_sn=True):
#                   super(PatchGAN_D,self).__init__()
#                   model=[]
#                   self.use_sn=use_sn
#                   #平面卷积
#                   model+=[nn.Conv2d(out_channels,hw,kernel_size=3,stride=1,padding=1,bias=True),
#                           nn.LeakyReLU(0.2,True)
#                   ]
#                   #下采样
#                   model+=[nn.Conv2d(hw,hw*2,kernel_size=3,stride=2,padding=1,bias=True),
#                               nn.LeakyReLU(0.2,True),nn.Conv2d(hw*2,hw*4,kernel_size=3,stride=1,padding=1,bias=True),
#                               norm(hw*4),nn.LeakyReLU(0.2,True),
#                               nn.Conv2d(hw*4,hw*4,kernel_size=3,stride=2,padding=1,bias=True),
#                               nn.LeakyReLU(0.2,True),nn.Conv2d(hw*4,hw*8,kernel_size=3,stride=1,padding=1,bias=True),
#                               norm(hw*8),nn.LeakyReLU(0.2,True),
#                               nn.Conv2d(hw*8,1,kernel_size=3,stride=1,padding=1),nn.Sigmoid()
#                               ]
#                   if self.use_sn:
#                         for i in range(len(model)):
#                             if isinstance(model[i], nn.Conv2d):
#                                 model[i] = spectral_norm(model[i]) 
#                   self.model=nn.Sequential(*model)
#           def forward(self,x):
#                     return self.model(x)
#判别器结构patchGAN
# class Discriminator(nn.Module):
#           def __init__(self):
#                   super(Discriminator_T,self).__init__()
#                   self.model=[PatchGAN_D()]
#                   self.model=nn.Sequential(*self.model)
                 
#           def forward(self,x):
#                 return self.model(x)
#           def init_train(self,grad=None):
#               for param in self.model.parameters():
#                   param.requires_grad = grad 

#VGG各种权重下载地址
model_weight_urls = {
    "vgg11": "https://download.pytorch.org/models/vgg11-8a719046.pth",
    "vgg13": "https://download.pytorch.org/models/vgg13-19584684.pth",
    "vgg16": "https://download.pytorch.org/models/vgg16-397923af.pth",
    "vgg19": "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth",
    "vgg11_bn": "https://download.pytorch.org/models/vgg11_bn-6002323d.pth",
    "vgg13_bn": "https://download.pytorch.org/models/vgg13_bn-abd245e5.pth",
    "vgg16_bn": "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth",
    "vgg19_bn": "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth",
}

#VGG19
class VGG19(nn.Module):
    def __init__(self,batch_norm=False,num_classes=1000):
        super(VGG19, self).__init__()
        self.cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512,
                    'M']
        self.batch_norm=batch_norm
        self.num_clases = num_classes
        self.features=self.make_layers(self.cfg,self.batch_norm)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
    def make_layers(self, cfg, batch_norm=False):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)
    def forward(self,x):
        module_list = list(self.features.modules())
        for l in module_list[1:27]:  # conv4_4
            x = l(x)
        return x
#SL-LIN
class content_struct(nn.Module):
    def __init__(self):
        super(content_struct, self).__init__()
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(128 * 16 * 16, 512)
#         self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
#         x = nn.functional.relu(self.conv1(x))
#         x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
#         x = x.view(-1, 128 * 16 * 16)
#         x = nn.functional.relu(self.fc1(x))
#         x = self.fc2(x)
        return x




In [5]:
from torch import nn
import torch
import time as t
from  tqdm import tqdm
from torch import optim
from torchvision import transforms
# from torchvision.transforms import InterpolationMode
from torchvision.utils import save_image
from torch.utils import data
from torch.nn.functional import interpolate
class MyGAN(object):
    def __init__(self, args):
        super().__init__()
        # 定义配置
        self.device = args.device
        self.result_dir = args.result_dir
        self.checkpoint_dir = args.checkpoint_dir
        self.dataset = args.dataset
        self.data_dir = args.data_dir
        self.test_dir = args.test_dir
        self.isTrain = args.isTrain
        self.isTest = args.isTest
        self.train_init = args.train_init
        self.epoch = args.epoch
        self.pre_epoch = args.pre_epoch
        self.cpu_count = args.cpu_count
        # 定义模型参数
        self.input_c = args.input_c
        self.hw = args.hw
        self.b1 = args.b1
        self.b2 = args.b2
        self.y1 = args.y1
        self.y2 = args.y2
        self.g_lr = args.g_lr
        self.d_lr = args.d_lr
        self.latent_dim = args.latent_dim
        self.pk = args.patch_size
        self.s = args.s
        self.decay_lr = args.decay_lr
        self.init_lr = args.init_lr
        self.batch_size = args.batch_size
        self.save_pred = args.save_pred
        # 模型权重参数w
        self.weight_content = args.weight_content
        self.weight_surface = args.weight_surface
        self.weight_testure = args.weight_testure
        self.weight_struct = args.weight_struct

        self.weight_style = args.weight_style
        self.weight_decay = args.weight_decay
        self.tv_weight = args.weight_tv
        # 定义模型
        self.G = Generator().to(self.device)
        self.D = Discriminator_S().to(self.device)
        self.D_patch = Discriminator_T().to(self.device)
        self.style_net=StyleEncoder().to(self.device)
        self.sct=utm().to(self.device)
        self.vgg19 = VGG19().to(self.device)
        self.vgg19.eval()
        self.p=content_struct().to(self.device)
        # 模型初始化
        self.G.apply(init_weights)
        self.D.apply(init_weights)
        self.D_patch.apply(init_weights)
        self.sct.apply(init_weights)
        self.style_net.apply(init_weights)
        self.vgg19.load_state_dict(torch.load('/kaggle/input/vgg19p/vgg19.pth'))
        # 定义优化器
        if self.train_init:
             self.optim_G = optim.Adam(self.G.parameters(), lr=self.init_lr, betas=(self.b1, self.b2))
        else:        
             self.optim_G = optim.Adam(self.G.parameters(), lr=self.g_lr, betas=(self.b1, self.b2))
        self.op_style_net=optim.Adam(self.style_net.parameters(), lr=self.d_lr, betas=(self.b1, self.b2))
        self.optim_sct = optim.Adam(self.sct.parameters(), lr=self.d_lr, betas=(self.b1, self.b2))
        self.optim_D = optim.Adam(self.D.parameters(), lr=self.d_lr, betas=(self.b1, self.b2))
        self.optim_D_Patch = optim.Adam(self.D_patch.parameters(), lr=self.g_lr, betas=(self.b1, self.b2))
        # 定义损失
        self.l1_loss = nn.L1Loss()
        self.huber = nn.SmoothL1Loss()
        self.gan_loss = nn.MSELoss()
        self.tv_loss = VariationLoss(1)
        self.lsty=nn.CrossEntropyLoss()
        self._rgb_to_yuv_kernel = torch.tensor([
            [0.299, -0.14714119, 0.61497538],
            [0.587, -0.28886916, -0.51496512],
            [0.114, 0.43601035, -0.10001026]
        ]).float().to(self.device)
        # 辅助器).to(self.device)
        self.gf = GuidedFilter()
        self.query = Queue()
        self.edge_exactor = CannyFilter().to(self.device)
        # 打印配置
        print("##### Information #####")
        print("# device : ", self.device)
        print(f"线程:{int(self.cpu_count)}")
        print("# dataset : ", self.dataset)
        print(f'# G_num :{print_network(self.G)}')
        print(f'# D_num :{print_network(self.D)}')
        print(f'# D_Patch_num :{print_network(self.D_patch)}')
        print("# batch_size : ", self.batch_size)
        print("# epoch : ", self.epoch)
        print("# pre epoch : ", self.pre_epoch)
        print("# init_train : ", self.train_init)
        print("# training image size [H, W] : ", self.hw)
        print("# content,style,suface,testure,color_weight,tv_weight: ", self.weight_content, self.weight_style,self.weight_surface,
              self.weight_testure,   self.weight_struct,self.tv_weight)
        print("# init_lr,g_lr,d_lr: ", self.init_lr, self.g_lr, self.d_lr)
    #生成假图.to(self.device)  

    # 读取数据集
    def load_data(self, epoch_test=False,high=False):
        self.mode = epoch_test
        self.high=high
        # 增强操作
        train_trans = [transforms.Resize(286),
               transforms.CenterCrop(256),
               transforms.RandomHorizontalFlip(0.5),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        test_trans = [transforms.Resize([512,512]),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        if self.mode and self.isTrain:
            data_loader = data.DataLoader(ImagePools(root=self.data_dir, trans=test_trans,mode=self.mode),
                                          batch_size=4,
                                          pin_memory=True,
                                          drop_last=True
                                          , num_workers=0)
        if self.isTrain and not self.mode:
            data_loader = data.DataLoader(ImagePools(root=self.data_dir, trans=train_trans),
                                          batch_size=self.batch_size, pin_memory=False,
                                          drop_last=True
                                          , num_workers=0)

        if self.isTest and self.mode:
            if self.high:
                 data_loader = data.DataLoader(ImagePools(root=self.data_dir, trans=test_trans,mode=self.mode,high=self.high), batch_size=1, num_workers=2)
            else:
                data_loader = data.DataLoader(ImagePools(root=self.data_dir, trans=test_trans,mode=self.mode), batch_size=1, num_workers=2)
        if self.isTest and not self.mode:
            data_loader = data.DataLoader(ImagePools(root=self.data_dir, trans=test_trans), batch_size=1, num_workers=2)    
        return data_loader


    # 加载灰度patch
    def load_patch(self, real, fake):
        # 进行灰度颜色改变
        real_gry= color_shift(real)
        fake_gry=color_shift(fake)
        return real_gry, fake_gry

    # conten loss
    def content_loss(self, fake, real):
        _,c,w,h=fake.shape
        out_con = self.l1_loss(fake,real)
        return out_con

    # dis loss
    def discriminator_loss(self, real, fake):
        real_loss = torch.mean(torch.square(real - 1.0))
        fake_loss = torch.mean(torch.square(fake))
        loss = real_loss + fake_loss
        return loss

    def generator_loss(self, fake):
        fake_loss = torch.mean(torch.square(fake - 1.0))
        return fake_loss
    # dis loss
    def discriminator_gram_loss(self, real,fake):
        real = gram(real)
        fake = gram(fake)
        real_loss = torch.mean(torch.square(real - 1.0))
        fake_loss = torch.mean(torch.square(fake))
        loss = real_loss + fake_loss
        return loss

    def generator_gram_loss(self, fake):
        fake = gram(fake)
        fake_loss = torch.mean(torch.square(fake - 1.0))
        return fake_loss

    # 训练
    def train(self):
        data_loader = self.load_data()
        count = len(data_loader)
       
#             self.classifier_optim.param_groups[0]['lr']-=self.lr/(self.iter//2)
        start_t = t.time()
        self.G.train()
        if self.train_init:
            print("=============================pre train phase==============================")
            for epoch in tqdm(range(self.pre_epoch)):
                for i, (x, y) in enumerate(data_loader):
                    x, y = x.to(self.device), y.to(self.device)
                    # 预训练阶段
                    requires_grad(self.D, False)
                    requires_grad(self.D_patch, False)
                    requires_grad(self.style_net, False)
                    self.optim_G.zero_grad()
                    self.optim_sct.zero_grad()
                    con_code= self.G.encoder(x)
                    con_code=self.sct(con_code,init=True)
                    fake1_img=self.G.decoders(con_code)
                    real_con = self.vgg19(x)
                    fake_con1 = self.vgg19(fake1_img)
                    con_loss_1=self.content_loss(fake_con1,real_con.detach())
                    #128
                    real_con2=interpolate(x,scale_factor=0.5,mode='bilinear')
                    real_con2=interpolate(real_con2,scale_factor=2,mode='bilinear')
                    fake_con2=interpolate(fake1_img,scale_factor=0.5,mode='bilinear')
                    fake_con2=interpolate(fake_con2, scale_factor=2, mode='bilinear')
                    con_loss_2=self.content_loss(fake_con2,real_con2.detach())
                    #64    
                    real_con3=interpolate(x,scale_factor=0.25,mode='bilinear')
                    real_con3=interpolate(real_con3,scale_factor=4,mode='bilinear')
                    fake_con3=interpolate(fake1_img,scale_factor=0.25,mode='bilinear')
                    fake_con3=interpolate(fake_con3, scale_factor=4, mode='bilinear')
                    con_loss_3= self.content_loss(fake_con3,real_con3.detach())
                    con_loss=(con_loss_1+con_loss_2+con_loss_3)/3*self.weight_content

                    con_loss.backward()
                    self.optim_G.step()
                    self.optim_sct.step()
                    end_epoch_t = t.time()
                    print(
                        f"epoch:[{epoch + 1}/{self.pre_epoch}],iter:[{i + 1}/{count}],loss_G:{con_loss},G_lr:{self.optim_G.param_groups[0]['lr']},"
                        f"time:{time_change(end_epoch_t - start_t)}")
                if epoch % self.save_pred == 0:
                    self.save_img1(epoch)
        else:
            print('==========================start train=====================================')
            for epoch in tqdm(range(self.epoch)):
                 # 学习率衰退
                if epoch>99:
                    self.optim_G.param_groups[0]['lr']-=0.0002/50
                    self.optim_D.param_groups[0]['lr']-=0.0002/50
                    self.optim_D_Patch.param_groups[0]['lr']-=0.0002/50
                    self.optim_sct.param_groups[0]['lr']-=0.0002/50
                    self.op_style_net.param_groups[0]['lr']-=0.0002/50
                for i, (x, y) in enumerate(data_loader):
                    x, y = x.to(self.device), y.to(self.device)
                    self.D.train(), self.D_patch.train()
                    # zero grident
                    self.optim_D.zero_grad()
                    self.optim_D_Patch.zero_grad()
                    self.op_style_net.zero_grad()
                    self.optim_sct.zero_grad()
                    # D
                    style_code=self.style_net(y)
                    content_code= self.G.encoder(x)
                    share_code=self.sct(content_code,style_code)
                    fake_img=self.G.decoders(share_code)
              # 减少模型震荡
                    # surface
                    gf_real_img_h =interpolate(y,scale_factor=2,mode='bilinear')
                    gf_fake_img_h =interpolate(fake_img,scale_factor=2,mode='bilinear')
                    gf_real_img_h = self.gf.guided_filter(gf_real_img_h, gf_real_img_h, r=5, eps=2e-1)
                    gf_fake_img_h = self.gf.guided_filter(gf_fake_img_h, gf_fake_img_h, r=5, eps=2e-1)
                    
                    d_real_logit=self.D(gf_real_img_h)
                    d_fake_logit=self.D(gf_fake_img_h.detach())
                    d_surface_loss= self.discriminator_loss(d_real_logit, d_fake_logit)
                    # testure
                    anime_gry_patch, fake_gry_patch= self.load_patch(gf_real_img_h, gf_fake_img_h)
#                     anime_gry_patch = interpolate(anime_gry_patch,scale_factor=2,mode='bilinear')
#                     fake_gry_patch = interpolate(fake_gry_patch,scale_factor=2,mode='bilinear')
                    real_patch_logit=self.D_patch(anime_gry_patch)
                    fake_patch_logit= self.D_patch(fake_gry_patch.detach())
                    d_testure_loss = self.discriminator_gram_loss(real_patch_logit,fake_patch_logit)
                    d_loss = (d_surface_loss + d_testure_loss) / 2
                    d_loss.backward()
                    self.optim_D.step()
                    self.optim_D_Patch.step()
                    # G
                    self.style_net.train()
                    self.sct.train()
                    self.optim_G.zero_grad()
                    self.op_style_net.zero_grad()
                    self.optim_sct.zero_grad()
                     #style  loss
                    style_code=self.style_net(y)
                    content_code= self.G.encoder(x)
                    share_code=self.sct(content_code,style_code)
                    fake1_img=self.G.decoders(share_code)
                    # surface
                    gf_fake_img_h1 =interpolate(fake1_img,scale_factor=2,mode='bilinear')
#                     y =interpolate(y,scale_factor=2,mode='bilinear')
                    gf_fake_img_h1 = self.gf.guided_filter(gf_fake_img_h1, gf_fake_img_h1, r=5, eps=2e-1)
                    
                    g_fake_logit1 = self.D(gf_fake_img_h1)
                    g_surface_loss =self.generator_loss(g_fake_logit1)*self.weight_surface
                    # testure
                    _, fake_gry_patch1 = self.load_patch(gf_fake_img_h1, gf_fake_img_h1)
#                     fake_gry_patch1 = interpolate(fake_gry_patch1,scale_factor=2,mode='bilinear')
                    fake_patch_logit=self.D_patch(fake_gry_patch1)
            
                    g_testure_loss = self.generator_gram_loss(fake_patch_logit)*self.weight_testure
                    # multi-level content loss
                    real_con = self.vgg19(x)
                    fake_con1 = self.vgg19(fake1_img)
                    con_loss_1=self.content_loss(fake_con1,real_con.detach())
                    #128
                    real_con2=interpolate(x,scale_factor=0.5,mode='bilinear')
                    real_con2=interpolate(real_con2,scale_factor=2,mode='bilinear')
                    fake_con2=interpolate(fake1_img,scale_factor=0.5,mode='bilinear')
                    fake_con2=interpolate(fake_con2, scale_factor=2, mode='bilinear')
                    con_loss_2=self.content_loss(fake_con2,real_con2.detach())
                    #64    
                    real_con3=interpolate(x,scale_factor=0.25,mode='bilinear')
                    real_con3=interpolate(real_con3,scale_factor=4,mode='bilinear')
                    fake_con3=interpolate(fake1_img,scale_factor=0.25,mode='bilinear')
                    fake_con3=interpolate(fake_con3, scale_factor=4, mode='bilinear')
                    con_loss_3= self.content_loss(fake_con3,real_con3.detach())
                    con_loss=(con_loss_1+con_loss_2+con_loss_3)/3*self.weight_content
                    # tv loss
                    tv_loss= self.tv_loss(fake1_img)
                     # color re LOSS
                    col_real_img = rgb_to_yuv(x, self._rgb_to_yuv_kernel)
                    col_fake_img = rgb_to_yuv(fake1_img, self._rgb_to_yuv_kernel)
                    col_loss = 10* (
                            self.l1_loss(col_real_img[:, 0, :, :], col_fake_img[:, 0, :, :]) + self.huber(
                            col_real_img[:, 1, :, :], col_fake_img[:, 1, :, :]) + \
                                self.huber(col_real_img[:, 2, :, :], col_fake_img[:, 2, :, :]))

                    g_loss = (g_surface_loss + g_testure_loss +con_loss + tv_loss +col_loss)/5
                    #                     else:
                    g_loss.backward()
                    self.optim_G.step()
                    self.op_style_net.step()
                    self.optim_sct.step()
                    end_epoch_t = t.time()
                    print(f"epoch:[{epoch + 1}/{self.epoch}],iter:[{i + 1}/{count}],loss_G:{g_loss},loss_d:{d_loss},G_lr:{self.optim_G.param_groups[0]['lr']},D_lr:{self.optim_D.param_groups[0]['lr']},time:{time_change(end_epoch_t - start_t)}")
                if (epoch + 1) % self.save_pred == 0:

                    with torch.no_grad():
                        self.save_model()
#                          self.save_img(epoch)
    def save_img1(self, epoch):
        test_sample_num =5
        self.G.eval()
        data_loader = self.load_data()
        for j in range(test_sample_num):
            for i, (x, y) in tqdm(enumerate(data_loader)):
                break
            x = x.to(self.device)
            y = y.to(self.device)
            content_code= self.G.encoder(x)
            fake_img1=self.G.decoders(content_code) # 生成假图
           
            image = torch.cat((x*0.5+0.5, fake_img1*0.5+0.5), axis=3)
            save_image(image, os.path.join(self.result_dir, self.dataset, 'img', f"train_{j}{epoch}.png"))
        print("训练集测试图像生成成功！")
        self.save_model()
        self.G.train(),self.style_net.train(),self.sct.train()

    def save_img(self, epoch):
        test_sample_num =1
        self.G.eval(),self.style_net.eval(),self.sct.eval()
        data_loader = self.load_data()
        for j in range(test_sample_num):
            for i, (x, y) in tqdm(enumerate(data_loader)):
                break
            x = x.to(self.device)
            y = y.to(self.device)
            rand_style = torch.randn([self.batch_size,self.hw,self.latent_dim,self.latent_dim]).to(self.device).requires_grad_()
            style_code=self.style_net(y)
            content_code= self.G.encoder(x)
            share_code=self.sct(content_code,style_code,rand_style)
            fake_img1=self.G.decoders(share_code) # 生成假图
            fake_img2 =self.gf.guided_filter(x, fake_img1, r=1)
            image = torch.cat((x*0.5+0.5, fake_img1*0.5+0.5,fake_img2*0.5+0.5), axis=3)
            save_image(image, os.path.join(self.result_dir, self.dataset, 'img', f"train_{j}{epoch}.png"))
        print("训练集测试图像生成成功！")
        test_loader = self.load_data(epoch_test=True)
        for i, (x, y) in tqdm(enumerate(test_loader)):
            break
        x = x.to(self.device)
        y = y.to(self.device)
#         style1_code,_ = self.style_net(lr_test)
        rand_style = torch.randn([4,self.hw,self.latent_dim,self.latent_dim]).to(self.device).requires_grad_()
        style_code=self.style_net(y)
        content_code= self.G.encoder(x)
        share_code=self.sct(content_code,style_code,rand_style)
        fake_img=self.G.decoders(share_code)
        lr_img2 =self.gf.guided_filter(x, fake_img, r=1)
        image = torch.cat((x*0.5+0.5, fake_img*0.5+0.5,lr_img2*0.5+0.5), axis=3)
        save_image(image, os.path.join(self.result_dir, self.dataset, 'img', f"test_{epoch}.png"))
        print("高分辨率测试集测试图像生成成功！")
        self.save_model()
        self.G.train(),self.style_net.train(),self.sct.train()

    # 保存模型
    def save_model(self):
        params = {}
        params["G"] = self.G.state_dict()
        params['sct']=self.sct.state_dict()
        params["style"] = self.style_net.state_dict()
        params["D"] = self.D.state_dict()
        params["D_patch"] = self.D_patch.state_dict()
        torch.save(params, os.path.join(self.result_dir, self.dataset, self.checkpoint_dir,
                                        f'checkpoint_{self.dataset}.pth'))
        print("保存模型成功！")

    # 加载模型
    def load_model(self):
        params = torch.load(self.test_dir)
        self.G.load_state_dict(params['G'])
        self.sct.load_state_dict(params['sct'])
        self.style_net.load_state_dict(params['style'])
        self.D.load_state_dict(params['D'])
        self.D_patch.load_state_dict(params['D_patch'])
        print("加载模型成功！")

    def test(self):
        self.load_model()
        test_sample_num =1
        self.G.eval(),self.style_net.eval(),self.sct.eval()
        data_loader = self.load_data()
        for j in range(test_sample_num):
            for i, (x, y) in tqdm(enumerate(data_loader)):
                x = x.to(self.device)
                y = y.to(self.device)
                # rand_style = torch.randn([2,self.hw,self.latent_dim,self.latent_dim]).to(self.device).requires_grad_()
                style_code=self.style_net(y)
                content_code= self.G.encoder(x)
                share_code=self.sct(content_code,style_code)
                fake_img=self.G.decoders(share_code) # 生成假图
                save_image(fake_img[0]*0.5+0.5 , os.path.join(self.result_dir, self.dataset, 'img', f'anime_lr{j}to{i}.png'))
        print("训练集测试图像生成成功！")
        # self.high_test()
    def  high_test(self):  
        self.load_model()
        data_loader = self.load_data(epoch_test=True,high=True)
        self.G.eval(),self.style_net.eval(),self.sct.eval()
        for i, (x, y) in tqdm(enumerate(data_loader)):
            x = x.to(self.device)
            y = y.to(self.device)
            # rand_style = torch.randn([1,self.hw,self.latent_dim,self.latent_dim]).to(self.device).requires_grad_()
            style_code=self.style_net(y)
            content_code= self.G.encoder(x)
            share_code=self.sct(content_code,style_code)
            fake_img=self.G.decoders(share_code)
            image = torch.cat((x*0.5+0.5, fake_img*0.5+0.5), axis=3)
            save_image(image, os.path.join(self.result_dir, self.dataset, 'img', f"test_high{i}.png"))
            print("高分辨率测试集测试图像生成成功！")
           

In [ ]:
import argparse
import os
def parse_args():
    desc='pytorch of My-CartoonGAN'
    parser=argparse.ArgumentParser(desc)
    parser.add_argument('--device',type=str,default='cuda',choices=['cuda','cpu'])
    parser.add_argument('--input_c', type=int, default=3)
    parser.add_argument('--epoch', type=int, default=150)
    parser.add_argument('--pre_epoch', type=int, default=20)
    parser.add_argument('--cpu_count', type=int, default=10)
    parser.add_argument('--init_lr',type=float,default=0.002)
    parser.add_argument('--g_lr', type=float, default=0.0002, help='learning rate for ADAM')
    parser.add_argument('--d_lr', type=float, default=0.0002, help='learning rate for ADAM')
    parser.add_argument('--decay_lr', type=float, default=0.0002, help='learning rate for ADAM')#3.000000000000006e-06
    parser.add_argument('--hw', type=int, default=256)
    parser.add_argument('--result_dir',type=str,default='results')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoint',
                        help='Name of checkpoint directory')
    parser.add_argument('--dataset', type=str, default='hayao')
    parser.add_argument('--data_dir',type=str,default='your data')
    parser.add_argument('--test_dir',type=str,default='your weight')
    parser.add_argument('--isTrain',type=bool,default=True)  #Train or Test
    parser.add_argument('--retrain', type=bool, default=False)
    parser.add_argument('--isTest', type=bool, default=False)
    parser.add_argument('--train_init', type=bool, default=False)
    parser.add_argument('--b1',type=int,default=0.5)
    parser.add_argument('--b2', type=int, default=0.999)
    parser.add_argument('--y1', type=int, default=1)
    parser.add_argument('--y2', type=int, default=10)
    parser.add_argument('--latent_dim', type=int, default=64)
    parser.add_argument('--patch_size', type=int, default=96)
    parser.add_argument('--s', type=int, default=48)
    parser.add_argument('--batch_size',type=int,default=4)
    parser.add_argument('--save_pred',type=int,default=1)
    parser.add_argument('--weight_content',type=float,default=2)#big
    parser.add_argument('--weight_struct',type=float,default=2)#
    parser.add_argument('--weight_surface', type=float, default=5)
    parser.add_argument('--weight_testure', type=float, default=5)
    parser.add_argument('--weight_classifer', type=float, default=0)
    parser.add_argument('--weight_tv', type=float, default=1)
    parser.add_argument('--weight_style', type=float, default=2)
    parser.add_argument('--weight_edge', type=float, default=0.001)
    parser.add_argument('--weight_decay', type=float, default=0.0001)
    return check_args(parser.parse_args(args=[]))
def check_args(args):
    check_folder(os.path.join(args.result_dir, args.dataset, 'checkpoint'))
    check_folder(os.path.join(args.result_dir, args.dataset, 'img'))
    check_folder(os.path.join(args.result_dir, args.dataset, 'test'))
    check_folder(os.path.join(args.result_dir, args.dataset, 'test','testA'))
    check_folder(os.path.join(args.result_dir, args.dataset, 'test','testB'))
    return args
def main():
   args=parse_args()
   gan=MyGAN(args)
   if args.isTrain:
       if args.retrain:
            gan.load_model()
       print(f"training on {args.device}")
       gan.train()
       print("train haved finished")
   if args.isTest:
       gan.test()
       print("test haved finished")
if __name__=="__main__":
    main()

##### Information #####
# device :  cuda
线程:10
# dataset :  hayao
# G_num :11600940
# D_num :287395
# D_Patch_num :389665
# batch_size :  4
# epoch :  30
# pre epoch :  20
# init_train :  False
# training image size [H, W] :  256
# content,style,suface,testure,color_weight,tv_weight:  2 2 5 5 2 1
# init_lr,g_lr,d_lr:  0.002 0.0002 0.0002
加载模型成功！
training on cuda
==========================start train=====================================


  0%|          | 0/30 [00:00<?, ?it/s]

epoch:[1/30],iter:[1/1611],loss_G:1.740273118019104,loss_d:0.9257938861846924,G_lr:0.0002,D_lr:0.0002,time:00h00m01s
epoch:[1/30],iter:[2/1611],loss_G:1.7380377054214478,loss_d:3.6916682720184326,G_lr:0.0002,D_lr:0.0002,time:00h00m02s
epoch:[1/30],iter:[3/1611],loss_G:1.8028160333633423,loss_d:0.818515419960022,G_lr:0.0002,D_lr:0.0002,time:00h00m03s
epoch:[1/30],iter:[4/1611],loss_G:1.8454296588897705,loss_d:0.9271384477615356,G_lr:0.0002,D_lr:0.0002,time:00h00m03s
epoch:[1/30],iter:[5/1611],loss_G:1.7906092405319214,loss_d:0.8936306238174438,G_lr:0.0002,D_lr:0.0002,time:00h00m04s
epoch:[1/30],iter:[6/1611],loss_G:1.510707139968872,loss_d:0.8382182121276855,G_lr:0.0002,D_lr:0.0002,time:00h00m05s
epoch:[1/30],iter:[7/1611],loss_G:1.1499974727630615,loss_d:0.7436254024505615,G_lr:0.0002,D_lr:0.0002,time:00h00m06s
epoch:[1/30],iter:[8/1611],loss_G:0.8589025735855103,loss_d:0.7108334302902222,G_lr:0.0002,D_lr:0.0002,time:00h00m06s
epoch:[1/30],iter:[9/1611],loss_G:0.8870100975036621,loss_d

In [ ]:
import os
import zipfile
def file2zip(packagePath, zipPath):
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()
if __name__ == "__main__":
    # 文件夹路径
    packagePath = 'results/hayao/img'
    zipPath = 'rcct50.zip' 
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")